# Settings

In [ ]:
# Set working directory

%cd /home/mlpanel-example-3/

In [ ]:
# Check Pipeline config 

from pprint import pprint
import yaml

# Look on pipelines config 
CONFIG_PATH = 'config/pipeline_config.yml'
config = yaml.load(open(CONFIG_PATH), Loader=yaml.FullLoader)

pprint(config)

# Prepare dataset 

In [ ]:
# Load data and extract features 

from src.pipelines.dataset_build import dataset_build

dataset_build(CONFIG_PATH)

In [ ]:
%%bash 

# Same with bash command

python src/pipelines/dataset_build.py \
    --config=config/pipeline_config.yml

In [ ]:
# check dataset 
!ls -l data/processed

# Split train/test 

In [ ]:
# Split data 

from src.pipelines.split_train_test import split_dataset

split_dataset(CONFIG_PATH)

In [ ]:
%%bash 

# Same with bash command

python src/pipelines/split_train_test.py \
    --config=config/pipeline_config.yml 

In [ ]:
# Check train and test datsets are created
!ls data/processed/

# Train model

In [ ]:
from src.pipelines.train import train_model

train_model(CONFIG_PATH)

In [ ]:
%%bash 

# Same with bash command

python src/pipelines/train.py \
    --config=config/pipeline_config.yml

In [ ]:
# model is created
!ls experiments/models/

# Evaluate model 

In [ ]:
# Evaluate model and log metrics to MLPanel 

from src.pipelines.evaluate import evaluate_model

evaluate_model(CONFIG_PATH)

In [ ]:
%%bash 

# Same with bash command

python src/pipelines/evaluate.py \
    --config=config/pipeline_config.yml

In [ ]:
# Check metrics file is created

!ls experiments

In [ ]:
!cat experiments/metrics.json

# Get predictions

## Deploy model

1. enter project Example3 in UI;
2. register model from experiment **example-3-experiment-1**;
3. deploy model with name **Example3Model** *on gcp"

## Load data

In [ ]:
import pandas as pd

test_csv_path = config['split_train_test']['test_csv']
df_predict = pd.read_csv(test_csv_path).drop('species', axis=1)
df_predict_json = df_predict.to_json(orient='table')

## Get id of deployment

In [ ]:
import requests

# Get deployments
deployments = requests.get('http://mlpanel:8089/deployments').json()

# Get deployments with models of Example 3
example3_deployments = [
    deployment for deployment in deployments if deployment.get('model_id') == 'Example3Model'
]

# Get the first deployment id
deployment_id = example3_deployments[0].get('id') if len(example3_deployments) > 0 else None
deployment_id

## Healthcheck deployment

In [ ]:
healthcheck = False

if deployment_id:
    healthcheck_response = requests.get(f'http://mlpanel:8089/deployments/{deployment_id}/ping')
    healthcheck = (healthcheck_response.status_code == 200)

healthcheck

## Make predict request

In [ ]:
predictions = None

if healthcheck:
    url = f'http://mlpanel:8089/deployments/{deployment_id}/predict'
    data = {'data': df_predict_json}
    response = requests.post(url, data)
    predictions = response.json()
    
predictions

## Example using *curl*

In [ ]:
if healthcheck:
    !curl -X POST "http://mlpanel:8089/deployments/{deployment_id}/predict" \
    -H "accept: application/json" \
    -H "Content-Type: multipart/form-data" \
    -F "data={df_predict_json.replace('"', '\\"')}"